In [2]:
!pip install imageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.4/315.4 kB 1.5 MB/s eta 0:00:001.6 MB/s eta 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import imageio
img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

/var/folders/vr/b837ndks0hd_73hq93kslgpc0000gp/T/ipykernel_13180/3731146654.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('../data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

In [15]:
import torch
img = torch.from_numpy(img_arr)
img.shape

torch.Size([720, 1280, 3])

In [16]:
img = img.permute(2,0,1)
img.shape

torch.Size([3, 720, 1280])

In [20]:
batch_size=3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)
batch.shape, batch.dtype

(torch.Size([3, 3, 256, 256]), torch.uint8)

In [41]:
import os 
data_dir = '../data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
            if os.path.splitext(name)[-1] == '.png']
print(filenames)

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    #print(img_t.shape)
    img_t = img_t.permute(2,0,1)
    #print(img_t.shape)
    img_t = img_t[:3]
    print(img_t.shape)
    batch[i] = img_t
    
batch.shape, batch.dtype

['cat1.png', 'cat2.png', 'cat3.png']
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])
torch.Size([3, 256, 256])


/var/folders/vr/b837ndks0hd_73hq93kslgpc0000gp/T/ipykernel_13180/2469954926.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


(torch.Size([3, 3, 256, 256]), torch.float32)

In [34]:
#normalize value by divinding by max
batch = batch.float()
batch.dtype
batch = batch/255.0

In [47]:
#normalize value by subtracting mean and divei by std dev
t = batch[:,0]
print(t.shape)
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:,c])
    std = torch.std(batch[:,c])
    batch[:,c] = (batch[:,c]-mean)/std
batch.shape, batch.dtype

torch.Size([3, 256, 256])


(torch.Size([3, 3, 256, 256]), torch.float32)

In [51]:
#vloume data of ct scan
dir_path = '../data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083'
vol_arr = imageio.volread(dir_path, 'DICOM')

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


In [55]:
vol = torch.from_numpy(vol_arr)
print(vol.shape)
vol = torch.unsqueeze(vol, 0)
vol.shape

torch.Size([99, 512, 512])


torch.Size([1, 99, 512, 512])

In [2]:
#tabular format data
import csv
import numpy as np
wine_path = "../data/p1ch4/tabular-wine/winequality-white.csv"
wine_arr = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)
wine_arr

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]],
      shape=(4898, 12), dtype=float32)

In [75]:
col_list = next(csv.reader(open(wine_path), delimiter=";"))
wine_arr.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [76]:
wineq = torch.from_numpy(wine_arr)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [80]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [82]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [85]:
target = target.long()
target, target.dtype

(tensor([6, 6, 6,  ..., 6, 7, 6]), torch.int64)

In [88]:
#one hot vector for the targets
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.shape

torch.Size([4898, 10])

In [90]:
target_onehot.scatter_( 1, target.unsqueeze(1), 1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [91]:
#normalize the dataset
data_mean = torch.mean(data, dim=0)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [92]:
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [94]:
data_normalized = (data - data_mean)/ data_var
data_normalized

tensor([[ 2.0392e-01, -8.1117e-01,  1.7622e+00,  ..., -8.2569e+00,
         -3.0593e+00, -1.1320e+00],
        [-7.7907e-01,  2.1417e+00,  3.9660e-01,  ...,  4.9003e+00,
          1.1761e-02, -6.6974e-01],
        [ 1.7486e+00,  1.7313e-01,  4.4933e+00,  ...,  3.1460e+00,
         -3.8271e+00, -2.7355e-01],
        ...,
        [-4.9822e-01, -3.7641e+00, -9.8453e+00,  ..., -8.6954e+00,
         -2.2916e+00, -7.3577e-01],
        [-1.9025e+00,  1.1574e+00, -2.3346e+00,  ...,  6.6546e+00,
         -8.4337e+00,  1.5093e+00],
        [-1.2004e+00, -6.7169e+00,  3.1278e+00,  ...,  3.1460e+00,
         -1.3040e+01,  8.4899e-01]])

In [97]:
# manual analysis to find the quality
bad_indexes = target <=3
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [98]:
# select bad data by using advanced indexing
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [105]:
bad_data = data[target <=3]
mid_data = data[(target>3) & (target <7)]
good_data = data[target>=7]
bad_mean = torch.mean(bad_data, 0)
mid_mean = torch.mean(mid_data, 0)
good_mean = torch.mean(good_data, 0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [106]:
total_sulphur_dioxide = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulphur_dioxide)
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [107]:
actual_indexes = target>5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [108]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)

In [18]:
import numpy as np
import torch
bikes_numpy = np.loadtxt("../data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
                         dtype=np.float32,
                         delimiter=",",
                         skiprows=1,
                         converters={1: lambda x: float(x[8:10])})
bikes_numpy[51:55]
col_list = next(csv.reader(open("../data/p1ch4/bike-sharing-dataset/hour-fixed.csv"), delimiter=","))
print(col_list)

['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']


In [13]:
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [19]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [21]:
daily_bikes = bikes.view(-1, 24 ,bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [22]:
# reshape for N * C * L ordering
#option 1
daily_bikes = daily_bikes.permute(0, 2, 1)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [23]:
#option2 
daily_bikes = bikes.view(-1, 24 ,bikes.shape[1])
daily_bikes = daily_bikes.transpose(1,2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [30]:
#encode the columns into categorical encoding 
#weather data to start with
first_day = bikes[:24].long()
first_day.shape

torch.Size([24, 17])

In [39]:
print(first_day[:,9].unsqueeze(1).shape)
weather_onehot = torch.zeros(first_day.shape[0], 4)
weather_onehot.scatter_(dim=1, index=first_day[:,9].unsqueeze(1) -1, value=1.0)
weather_onehot

torch.Size([24, 1])


tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [38]:
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [42]:
torch.cat((bikes[:24], weather_onehot), 1) [:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [44]:
bikes[:1], weather_onehot[:1]

(tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
           0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
          16.0000]]),
 tensor([[1., 0., 0., 0.]]))

In [45]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [49]:
daily_weather_onehot.scatter_(dim=1, index= daily_bikes[:,9,:].long().unsqueeze(1) -1, value=1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [50]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), 1)
daily_bikes.shape

torch.Size([730, 21, 24])

In [52]:
#temp we can scale
temp = daily_bikes[:,10,:]
temp_min = torch.min(temp)
temp_max = torch.max(temp)

daily_bikes [:, 10, :] = (daily_bikes [:, 10, :] - temp_min)/ (temp_max - temp_min)

In [53]:
#other way via mean and std dev
temp = daily_bikes[:,10,:]
temp_mean = torch.mean(temp)
temp_std = torch.var(temp)
daily_bikes [:, 10, :] = (daily_bikes [:, 10, :] - temp_mean)/ temp_std

In [54]:
# textembeddings
with open('../data/p1ch4/jane-austen/1342-0.txt', encoding='utf8') as f:
    text =f.read()

In [56]:
lines = text.split('\n')
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [67]:
#char embeddings one hot
letter_t = torch.zeros(len(line), 128)
letter_t.shape, letter_t.dtype

(torch.Size([70, 128]), torch.float32)

In [69]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    #print(letter_index)
    letter_t[i][letter_index] = 1.0
letter_t

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [72]:
#word embeddings 
def clean_words(input_str):
    punctuations='.,;:"!“?_-'
    word_list = input_str.lower().replace('\n', ' ').split()
    word_list = [word.strip(punctuations) for word in word_list]
    return word_list

In [73]:
words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [81]:
words_list = sorted(set(clean_words(text)))
words2indexdict = {word:i for (i, word) in enumerate(words_list)}
len(words2indexdict), words2indexdict['impossible']

(8267, 3848)

In [98]:
#word tensor
word_t = torch.zeros(len(words_in_line), len(words2indexdict))
for i, word in enumerate(words_in_line):
    wordindex = words2indexdict[word]
    word_t[i][wordindex] =1.0
    print('{:2} {:4} {}'.format(i, wordindex, word))
    
word_t.shape

 0 3848 impossible
 1 4928 mr
 2  899 bennet
 3 3848 impossible
 4 8049 when
 5 3759 i
 6  449 am
 7 5081 not
 8  249 acquainted
 9 8126 with
10 3637 him


torch.Size([11, 8267])

In [99]:
word_t = torch.zeros(len(clean_words(text)), len(words2indexdict))
for i, word in enumerate(clean_words(text)):
    wordindex = words2indexdict[word]
    word_t[i][wordindex] =1.0
    #print('{:2} {:4} {}'.format(i, wordindex, word))
    
word_t.shape

torch.Size([124592, 8267])

In [100]:
print(word_t[0])

tensor([0., 0., 0.,  ..., 0., 0., 1.])
